***GENERATED CODE FOR join PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:


def QueryEditor(spark, joindataobj):
    for source in joindataobj["sourcesForQuery"]:
        sourcename = source['sourcename']
        source['sourcename'] = spark.read.options(header='true', inferschema='true', delimiter=',')\
            .csv('dbfs:' + source['sourcepath'])
        source['sourcename'].createOrReplaceTempView(sourcename)
    joinedDf = spark.sql(joindataobj['querytext'])
    df = removeIfDuplicates(joinedDf)
    print(display(df.limit(2).toPandas()))
    return df


def removeIfDuplicates(joinedDf):
    cols_new = []
    seen = set()
    for c in joinedDf.columns:
        cols_new.append('{}_dup'.format(c) if c in seen else c)
        seen.add(c)
    df = joinedDf.toDF(*cols_new).select(*
                                         [c for c in cols_new if not c.endswith('_dup')])
    return df


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'price_sum', 'threshold': 259059.83388076647, 'transformation_label': 'Binarizer'}], 'feature': 'price_sum', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '4365', 'mean': '259059.83', 'stddev': '1465702.91', 'min': '81.57', 'max': '1.5953485738516184E7', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'price_sum'}, {'feature_label': 'price_sum', 'threshold': 259059.83388076647, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('price_sum')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run joinHooks.ipynb
try:
	#sourcePreExecutionHook()
		coupon = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/demo/coupon.csv', 'filename': 'coupon.csv', 'token': '', 'delimiter': ','}")


		campaign = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/demo/campaign.csv', 'filename': 'campaign.csv', 'token': '', 'delimiter': ','}")


		custdemo = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/demo/cust_demo.csv', 'filename': 'cust_demo.csv', 'token': '', 'delimiter': ','}")


		train = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/demo/train.csv', 'filename': 'train.csv', 'token': '', 'delimiter': ','}")



	transactions = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/demo/transactions.csv', 'filename': 'transactions.csv', 'token': '', 'delimiter': ','}")
	#sourcePostExecutionHook(transactions)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run joinHooks.ipynb
try:
	#operationPreExecutionHook()

	joindata = QueryEditor(spark, {"querytext": "SELECT *\r\nFROM train\r\nJOIN coupon\r\nON train.coupon_id = coupon.coupon_id\r\nJOIN campaign\r\nON train.campaign_id = campaign.campaign_id\r\nJOIN cust_demo\r\nON train.customer_id = cust_demo.customer_id\r\nJOIN transactions\r\nON cust_demo.customer_id = transactions.customer_id", "sourcesForQuery": [{"sourcename": "coupon", "sourcepath": "/FileStore/platform/demo/coupon.csv", "sourceid": "5f5f67fc3dc8e5d673385515"}, {"sourcename": "campaign", "sourcepath": "/FileStore/platform/demo/campaign.csv", "sourceid": "5f5f67fe3dc8e5d673385516"}, {"sourcename": "train", "sourcepath": "/FileStore/platform/demo/train.csv", "sourceid": "5f5f68033dc8e5d673385518"}, {"sourcename": "cust_demo", "sourcepath": "/FileStore/platform/demo/cust_demo.csv", "sourceid": "5f5f68003dc8e5d673385517"}, {"sourcename": "transactions", "sourcepath": "/FileStore/platform/demo/transactions.csv", "sourceid": "5f5f68063dc8e5d673385519"}], "fieldsForQuery": [{"sourcename": "coupon", "fieldname": "coupon_id"}, {"sourcename": "campaign", "fieldname": "campaign_id"}, {"sourcename": "train", "fieldname": "id"}, {"sourcename": "train", "fieldname": "campaign_id"}, {"sourcename": "train", "fieldname": "coupon_id"}, {"sourcename": "train", "fieldname": "customer_id"}, {"sourcename": "cust_demo", "fieldname": "customer_id"}, {"sourcename": "transactions", "fieldname": "customer_id"}]})
	#operationPostExecutionHook(joindata)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run joinHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(joindata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "568.35", "stddev": "330.46", "min": "1", "max": "1115", "missing": "0"}, "updatedLabel": "coupon_id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "customer_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "787.03", "stddev": "465.64", "min": "1", "max": "1581", "missing": "0"}, "updatedLabel": "customer_id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "redemption_status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "0.01", "stddev": "0.12", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "redemption_status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "1089.0", "stddev": "1372.99", "min": "1", "max": "5391", "missing": "0"}, "updatedLabel": "brand"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand_type", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "0.21", "stddev": "0.41", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "brand_type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "category", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "7.47", "stddev": "2.55", "min": "1", "max": "16", "missing": "0"}, "updatedLabel": "category"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "-1683.5", "stddev": "8779.4", "min": "-113627.33999999987", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "cd_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "-2.31", "stddev": "5.95", "min": "-71.24", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "coupon_discount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_used", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "49.66", "stddev": "247.31", "min": "0.0", "max": "2661.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "coupon_used"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "item_counts", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "106.03", "stddev": "765.86", "min": "1", "max": "11813", "missing": "0"}, "updatedLabel": "item_counts"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_customers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "17.39", "stddev": "13.45", "min": "1.0", "max": "118.57142857142857", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "no_of_customers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "od_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "-41452.83", "stddev": "248149.61", "min": "-3259657.797895498", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "od_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "other_discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "-20.02", "stddev": "18.79", "min": "-481.4993333333333", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "other_discount"}, {"transformationsData": [{"feature_label": "price_sum", "threshold": 259059.83388076647, "transformation_label": "Binarizer"}], "feature": "price_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "259059.83", "stddev": "1465702.91", "min": "81.57", "max": "1.5953485738516184E7", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "price_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "qu_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "3106.89", "stddev": "17983.57", "min": "1.0", "max": "207579.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "qu_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "quantity", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "1.18", "stddev": "0.27", "min": "1.0", "max": "4.096091911309303", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "selling_price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "160.28", "stddev": "139.2", "min": "13.416928746729855", "max": "1235.863616557734", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "selling_price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "t_counts", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "2403.99", "stddev": "13666.26", "min": "1.0", "max": "158788.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "t_counts"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_discount_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "-22.33", "stddev": "20.42", "min": "-508.808", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_discount_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_discount_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "-43136.33", "stddev": "255067.74", "min": "-3344221.267895495", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_discount_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_type", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "0.28", "stddev": "0.45", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "campaign_type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_duration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "44.76", "stddev": "6.32", "min": "32", "max": "63", "missing": "0"}, "updatedLabel": "campaign_duration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "age_range", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "2.46", "stddev": "1.26", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "age_range"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "marital_status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "0.41", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "marital_status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rented", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "0.05", "stddev": "0.23", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "rented"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "family_size", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "2.22", "stddev": "1.22", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "family_size"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_children", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "0.62", "stddev": "0.98", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "no_of_children"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "income_bracket", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "4.79", "stddev": "2.36", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "income_bracket"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "15.2", "stddev": "8.62", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "day"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dow", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "3.24", "stddev": "1.69", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "dow"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_items", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "669.21", "stddev": "334.1", "min": "97", "max": "2088", "missing": "0"}, "updatedLabel": "no_of_items"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "4.91", "stddev": "2.17", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cdd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "-902.52", "stddev": "1528.41", "min": "-11937.8", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "cdd_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "customer_id_count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "1198.77", "stddev": "683.87", "min": "144", "max": "4134", "missing": "0"}, "updatedLabel": "customer_id_count"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "odd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4365", "mean": "-16329.3", "stddev": "10016.81", "min": "-62321.37241", "max": "-873.175", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "odd_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "qa_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4365", "mean": "183155.43", "stddev": "274610.32", "min": "179", "max": "2617544", "missing": "0"}, "updatedLabel": "qa_sum"}, {"feature": "price_sum_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "4365", "mean": "0.09", "stddev": "0.29", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "price_sum_binarizer_trans..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run joinHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["coupon_id", "customer_id", "brand", "brand_type", "category", "cd_sum", "coupon_discount", "coupon_used", "item_counts", "no_of_customers", "od_sum", "other_discount", "price_sum_binarizer", "qu_sum", "quantity", "selling_price", "t_counts", "total_discount_mean", "total_discount_sum", "campaign_type", "campaign_duration", "age_range", "marital_status", "rented", "family_size", "no_of_children", "income_bracket", "day", "dow", "no_of_items", "month", "cdd_sum", "customer_id_count", "odd_sum", "qa_sum"], "redemption_status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

